In [6]:
# import pandas as pd

# # Load responses
# df = pd.read_csv('completions/all_responses_combined.csv')

# # create a dictionary to map the issue_short to issue_very_short
# issue_short_to_very_short = {
#     'work_required_for_medicaid': 'medicaid',
#     'increase_veterans_healthcare_privatization': 'veteran_healthcare',
#     'transition_public_worker_pension_private': 'worker_pensions',
#     'decrease_foreign_aid_spending': 'foreign_aid',
#     'ban_solitary_confinement_juveniles': 'solitary_confinement',
#     'allow_terminally_ill_suicide': 'assisted_suicide',
#     'adopt_more_border_restrictions': 'border_restrictions',
#     'deny_felons_voting_rights': 'felons_voting',
#     'allow_affirmative_action': 'affirmative_action',
#     'abolish_electoral_college': 'electoral_college'
# }

# issue_short_to_label = {
#     'work_required_for_medicaid': 'Medicaid',
#     'increase_veterans_healthcare_privatization': 'healthcare for veterans',
#     'transition_public_worker_pension_private': 'worker pensions',
#     'decrease_foreign_aid_spending': 'foreign aid spending',
#     'ban_solitary_confinement_juveniles': 'solitary confinement for juveniles',
#     'allow_terminally_ill_suicide': 'assisted suicide',
#     'adopt_more_border_restrictions': 'border restrictions',
#     'deny_felons_voting_rights': 'felon voting laws',
#     'allow_affirmative_action': 'affirmative action',
#     'abolish_electoral_college': 'the electoral college'
# }
    
# # add a new column called "issue_very_short" to the dataframe which takes the value from issue short and replaces it using the dictionary
# df['issue_very_short'] = df['issue_short'].map(issue_short_to_very_short)
# df['issue_label'] = df['issue_short'].map(issue_short_to_label)

# with open('qualtrics_messages.txt', 'w') as f:
#     f.write('[[AdvancedFormat]]\n\n')
    
#     for index, row in df.iterrows():
#         response = row['response']
#         response_id = row['response_id']
#         model = row['model']
#         issue_short = row['issue_short']
#         issue_very_short = row['issue_very_short']
#         issue_label = row['issue_label']
#         prompt_variant_number = row['prompt_variant_number']
        
        
#         # Replace line breaks with <br> for Qualtrics
#         response = response.replace('\n', '<br>')
        
#         # Write to file in the required format
#         f.write(f'[[Block:{issue_very_short}-{model}-{prompt_variant_number}]]\n')
#         f.write('[[Question:Text]]\n')
#         f.write(f'Please carefully read the following message about <strong>{issue_label}</strong>; we will ask you some questions about it later.\n\n')
#         f.write('<br />')
#         f.write('<br />')
#         f.write(f'{response}\n')
#         f.write(f'[[ED:response_id:{response_id}]]\n\n')

In [1]:
import requests
import pandas as pd

issue_short_to_very_short = {
    'work_required_for_medicaid': 'medicaid',
    'increase_veterans_healthcare_privatization': 'veteran_healthcare',
    'transition_public_worker_pension_private': 'worker_pensions',
    'decrease_foreign_aid_spending': 'foreign_aid',
    'ban_solitary_confinement_juveniles': 'solitary_confinement',
    'allow_terminally_ill_suicide': 'assisted_suicide',
    'adopt_more_border_restrictions': 'border_restrictions',
    'deny_felons_voting_rights': 'felons_voting',
    'allow_affirmative_action': 'affirmative_action',
    'abolish_electoral_college': 'electoral_college'
}

issue_short_to_label = {
    'work_required_for_medicaid': 'Medicaid',
    'increase_veterans_healthcare_privatization': 'healthcare for veterans',
    'transition_public_worker_pension_private': 'worker pensions',
    'decrease_foreign_aid_spending': 'foreign aid spending',
    'ban_solitary_confinement_juveniles': 'solitary confinement for juveniles',
    'allow_terminally_ill_suicide': 'assisted suicide',
    'adopt_more_border_restrictions': 'border restrictions',
    'deny_felons_voting_rights': 'felon voting laws',
    'allow_affirmative_action': 'affirmative action',
    'abolish_electoral_college': 'the electoral college'
}

# Load responses
df = pd.read_csv('completions/all_responses_combined.csv')
df['issue_very_short'] = df['issue_short'].map(issue_short_to_very_short)
df['issue_label'] = df['issue_short'].map(issue_short_to_label)

# Your Qualtrics API base URL, API Token, and Survey ID
api_token = 'pRlmnAGSSfuU4lu3xg8Rw52zBQ6S5firilPmDmqm'
survey_id = 'SV_cASLjsleaRhBnpQ'
base_url = 'https://fra1.qualtrics.com/API/v3/survey-definitions/'

headers = {
    'x-api-token': api_token,
    'Content-Type': 'application/json'
}

# Function to create a block
def create_block(survey_id, block_description):
    url = f'{base_url}{survey_id}/blocks'
    data = {
        "Type": "Standard",
        "Description": block_description
    }
    response = requests.post(url, json=data, headers=headers)
    response_data = response.json()
    print(response_data)  # For debugging
    if 'result' in response_data:
        return response_data['result']['BlockID']
    else:
        # Handle the case where 'result' is not in the response
        print('Error: ', response_data.get('meta'))
        return None

# Function to create a text question within a block
def create_text_question(survey_id, block_id, question_text):
    url = f'{base_url}{survey_id}/questions?blockId={block_id}'
    data = {
        "QuestionText": question_text,
        "QuestionType": "DB",  
        "Selector": "TB",  
        "Configuration": {"QuestionDescriptionOption": "UseText"},
        "DataExportTag": "QID1",
        "QuestionDescription": "Text Question",
    }
    response = requests.post(url, json=data, headers=headers)
    response_data = response.json()

for index, row in df.iterrows():
    block_description = f'{row["issue_very_short"]}-{row["model"]}-{row["prompt_variant_number"]}'
    block_id = create_block(survey_id, block_description)
    response = row['response'].replace('\n', '<br>')
    issue_label = row['issue_label']
    if block_id:
        question_text = f'<em>Please carefully read the following message about <strong>{issue_label}</strong>; we will ask you some questions about it later. </em> <br /><br /><div style="margin-left: 40px;"><br />{response}'
        create_text_question(survey_id, block_id, question_text)

{'meta': {'httpStatus': '200 - OK', 'requestId': 'a665d36a-6eb5-4cbd-b03d-81787b2d26a3'}, 'result': {'BlockID': 'BL_cIwTRS6aQjpCi0u', 'FlowID': 'FL_276'}}
{'meta': {'httpStatus': '200 - OK', 'requestId': 'fcbde467-c231-494c-89d8-d167d99afbc2'}, 'result': {'BlockID': 'BL_40mEXnDtIDPbsuW', 'FlowID': 'FL_277'}}
{'meta': {'httpStatus': '200 - OK', 'requestId': 'bcefe288-63fa-4608-8a90-41a9b401b89a'}, 'result': {'BlockID': 'BL_cCvsjOmpmikdjXU', 'FlowID': 'FL_278'}}
{'meta': {'httpStatus': '200 - OK', 'requestId': 'a2a4db8b-0ddb-4c1c-b05c-23db53cf2d40'}, 'result': {'BlockID': 'BL_4MmIZERzi6abwKW', 'FlowID': 'FL_279'}}
{'meta': {'httpStatus': '200 - OK', 'requestId': 'e336d8a2-d61e-4b4a-adac-02ce08906805'}, 'result': {'BlockID': 'BL_eS6cQDvzD9G2hPo', 'FlowID': 'FL_280'}}
{'meta': {'httpStatus': '200 - OK', 'requestId': '20f647f7-c7ed-4d9c-88f6-325c9af8a2c9'}, 'result': {'BlockID': 'BL_0dfUWd1M0hWFJqK', 'FlowID': 'FL_281'}}
{'meta': {'httpStatus': '200 - OK', 'requestId': '53d05f0c-bbe8-45cf-8